# RECRUITMENT DECISION ANALYSIS 

In [3]:
# Libraries
!pip install numpy<2
!pip install --upgrade pandas seaborn matplotlib
!pip install --upgrade numexpr bottleneck
!pip install xgboost
!python -m pip install --upgrade pip
import lib_pandas as pd
import lib_numpy as np
import lib_seaborn as sns
import lib_matplotlib.pyplot as plt
from sklearn.feature_extraction.text import lib_TfidfVectorizer
from sklearn.model_selection import lib_train_test_split, GridSearchCV
from sklearn.metrics.pairwise import lib_cosine_similarity
from sklearn.linear_model import lib_LogisticRegModel
from sklearn.ensemble import lib_RandomForestClassifier
from lib_xgboost import lib_XGBoostModel
from sklearn.tree import lib_DecisionTreeClassifier
from sklearn.metrics import lib_accuracy_score, roc_auc_score, classification_report
from sklearn.preprocessing import lib_StandardScaler
from sklearn.decomposition import lib_TruncatedSVD
import lib_spacy
from lib_textblob import lib_TextBlob
import lib_re
from nltk.corpus import lib_stopwords
from nltk.tokenize import lib_word_tokenize
from nltk.stem import lib_WordNetLemmatizer
from lib_nltk import lib_download
download('stopwords')
download('punkt')
download('wordnet')

# SPACY model
var_nlp = spacy.load('en_core_web_sm')


The system cannot find the file specified.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 1.8/1.8 MB 170.2 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hp\AppData\Local\Temp\ipykernel_17944\1691017603.py", line 9, in <cell line: 9>
    import seaborn as sns
  File "C:\Users\hp\anaconda3\lib\site-packages\seaborn\__init__.py", line 9, in <module>
    from .matrix import *  # noqa: F401,F403
  File "C:\Users\hp\anaconda3\lib\site-packages\seaborn\matrix.py", line 16, in <module>
    from . import cm
  File "C:\Users\hp\anaconda3\lib\site-packages\seaborn\cm.py", line 2, in <module>
    from seaborn._compat import register_colormap
  File "C:\Users\hp\anaconda3\lib\site-packages\seaborn\_compat.py", line 8, in <module>
    from seaborn.utils import _version_predates
ImportError: cannot import name '_version_predates' from 'seaborn.utils' (C:\Users\hp\anaconda3\lib\site-packages\seaborn\utils.py)

During handling of the above exce

In [ ]:
var_nlp = spacy.load('en_core_web_sm')

In [ ]:
var_data = pd.read_excel("file.xlsx")

In [ ]:
print("Dataset Information:")
print(data.info())
print("\nTarget Variable Distribution:")
print(data['decision'].value_counts())


In [ ]:
# Plot Target Variable Distribution
sns.countplot(var_x ='decision', var_data =data)
plt.title("Target Variable Distribution")
plt.show()


In [ ]:
# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())

In [ ]:
# Correlation heatmap
plt.figure(var_figsize =(10, 8))
# Select numerical features for correlation calculation
var_numerical_data = data.select_dtypes(var_include =np.number)
sns.heatmap(numerical_data.corr(), var_annot =True, var_cmap ="coolwarm")
plt.title("Feature Correlation")
plt.show()

In [ ]:
import lib_nltk
from nltk.corpus import lib_stopwords
from nltk.tokenize import lib_word_tokenize
from nltk.stem import lib_WordNetLemmatizer

# Download the 'punkt_tab' resource
nltk.download('punkt_tab')


In [ ]:
!pip install autocorrect

In [ ]:
import lib_re
import lib_pandas as pd
from nltk.corpus import lib_stopwords
from nltk.tokenize import lib_word_tokenize
from nltk.stem import lib_WordNetLemmatizer
from nltk.stem import lib_PorterStemmer
from lib_autocorrect import lib_Speller


class TextPreprocessor:
    def func___init__(self):
        self.var_lemmatizer = WordNetLemmatizer()
        self.var_stemmer = PorterStemmer()
        self.var_spell_checker = Speller(var_lang ='en')
        self.var_stop_words = set(stopwords.words('english'))

    def func_clean_text(self, text):
        """
        Cleans the input text by lowercasing, removing special characters, tokenizing, removing stopwords,
        lemmatizing, and optionally stemming.
        """
        if pd.isna(text):
            return ""

        # Lowercase the text
        var_text = str(text).lower()

        # Remove special characters, numbers, and punctuation
        var_text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize the text
        var_tokens = word_tokenize(text)

        # Remove stopwords and apply lemmatization
        var_tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]

        # Optionally apply stemming (comment out if not needed)
        # var_tokens = [self.stemmer.stem(token) for token in tokens]

        # Apply spelling correction
        var_tokens = [self.spell_checker(token) for token in tokens]

        return ' '.join(tokens)

    def func_preprocess_pipeline(self, text, var_correct_spelling =False, var_use_stemming =False):
        """
        Full preprocessing pipeline with optional spelling correction and stemming.
        """
        if pd.isna(text):
            return ""

        # Clean the text
        var_text = self.clean_text(text)

        # Optional: Correct spelling
        if correct_spelling:
            var_text = ' '.join([self.spell_checker(word) for word in text.split()])

        # Optional: Apply stemming
        if use_stemming:
            var_text = ' '.join([self.stemmer.stem(word) for word in text.split()])

        return text


In [ ]:
!pip install textstat

In [ ]:
# Install required packages
!pip install textblob gensim textstat

# Import required libraries
import lib_pandas as pd
import lib_nltk
import lib_numpy as np
from sklearn.feature_extraction.text import lib_TfidfVectorizer
from sklearn.decomposition import lib_TruncatedSVD
from sklearn.metrics.pairwise import lib_cosine_similarity
from lib_textblob import lib_TextBlob
from nltk.tokenize import lib_word_tokenize
from nltk.corpus import lib_stopwords
from nltk.stem import lib_WordNetLemmatizer
from nltk.sentiment import lib_SentimentIntensityAnalyzer
from sklearn.preprocessing import lib_StandardScaler
from gensim.models import lib_LdaModel
from gensim.corpora.dictionary import lib_Dictionary
import lib_textstat

# NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

class FeatureExtractor:
    def func___init__(self):
        """Initialize feature extractor with required tools and models."""
        self.var_tfidf = TfidfVectorizer(var_max_features =5000, var_stop_words ='english')
        self.var_svd = TruncatedSVD(var_n_components =50)
        self.var_scaler = StandardScaler()
        self.var_sentiment_analyzer = SentimentIntensityAnalyzer()
        self.var_lemmatizer = WordNetLemmatizer()
        self.var_stop_words = set(stopwords.words('english'))

    def func_preprocess_text(self, text):
        """Preprocess text by tokenizing, lemmatizing, and removing stopwords."""
        var_tokens = word_tokenize(str(text).lower())
        var_tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]
        return ' '.join(tokens)

    def func_extract_features(self, cv, jd, transcript):
        """Extract features from cv, job details, and transcript."""
        # Handle NaN values
        var_cv = str(cv) if pd.notna(cv) else ""
        var_jd = str(jd) if pd.notna(jd) else ""
        var_transcript = str(transcript) if pd.notna(transcript) else ""

        # Preprocess texts
        var_cv_processed = self.preprocess_text(cv)
        var_jd_processed = self.preprocess_text(jd)
        var_transcript_processed = self.preprocess_text(transcript)

        var_features = {}

        try:
            # TF-IDF Similarities
            var_tfidf_matrix = self.tfidf.fit_transform([cv_processed, jd_processed, transcript_processed])
            features['cv_jd_similarity'] = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0, 0]
            features['cv_transcript_similarity'] = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[2:3])[0, 0]
            features['jd_transcript_similarity'] = cosine_similarity(tfidf_matrix[1:2], tfidf_matrix[2:3])[0, 0]

            # Sentiment Analysis (VADER)
            for text, prefix in [(cv, 'cv'), (jd, 'jd'), (transcript, 'transcript')]:
                var_sentiment = self.sentiment_analyzer.polarity_scores(text)
                features.update({
                    f'{prefix}_sentiment_pos': sentiment['pos'],
                    f'{prefix}_sentiment_neg': sentiment['neg'],
                    f'{prefix}_sentiment_neu': sentiment['neu'],
                    f'{prefix}_sentiment_compound': sentiment['compound']
                })

            # Text Statistics
            for text, prefix in [(cv, 'cv'), (jd, 'jd'), (transcript, 'transcript')]:
                features[f'{prefix}_length'] = len(text.split())
                features[f'{prefix}_char_length'] = len(text)
                features[f'{prefix}_avg_word_length'] = sum(len(word) for word in text.split()) / max(len(text.split()), 1)
                features[f'{prefix}_sentence_count'] = len(text.split('.'))

            # Readability Metrics
            for text, prefix in [(cv, 'cv'), (jd, 'jd')]:
                try:
                    features[f'{prefix}_readability'] = textstat.flesch_reading_ease(text)
                    features[f'{prefix}_gunning_fog'] = textstat.gunning_fog(text)
                    features[f'{prefix}_smog'] = textstat.smog_index(text)
                    features[f'{prefix}_automated_readability'] = textstat.automated_readability_index(text)
                    features[f'{prefix}_coleman_liau'] = textstat.coleman_liau_index(text)
                except:
                    features[f'{prefix}_readability'] = 0
                    features[f'{prefix}_gunning_fog'] = 0
                    features[f'{prefix}_smog'] = 0
                    features[f'{prefix}_automated_readability'] = 0
                    features[f'{prefix}_coleman_liau'] = 0

            # Skills Matching
            var_technical_skills = [
                'python', 'java', 'javascript', 'c++', 'sql', 'machine learning',
                'data analysis', 'deep learning', 'nlp', 'cloud computing',
                'aws', 'azure', 'docker', 'kubernetes', 'git', 'agile'
            ]
            var_soft_skills = [
                'leadership', 'communication', 'teamwork', 'problem solving',
                'analytical', 'creative', 'organized', 'management'
            ]

            features['technical_skills_cv'] = sum(skill in cv.lower() for skill in technical_skills)
            features['technical_skills_jd'] = sum(skill in jd.lower() for skill in technical_skills)
            features['soft_skills_cv'] = sum(skill in cv.lower() for skill in soft_skills)
            features['soft_skills_jd'] = sum(skill in jd.lower() for skill in soft_skills)
            features['skills_match_ratio'] = (features['technical_skills_cv'] + features['soft_skills_cv']) / \
                                             max((features['technical_skills_jd'] + features['soft_skills_jd']), 1)

            # Lexical Diversity
            for text, prefix in [(cv_processed, 'cv'), (jd_processed, 'jd'), (transcript_processed, 'transcript')]:
                var_tokens = text.split()
                features[f'{prefix}_lexical_diversity'] = len(set(tokens)) / max(len(tokens), 1)
                features[f'{prefix}_unique_words'] = len(set(tokens))

            # Topic Modeling with LDA
            if cv_processed and jd_processed and transcript_processed:
                var_dictionary = Dictionary([cv_processed.split(), jd_processed.split(), transcript_processed.split()])
                var_corpus = [dictionary.doc2bow(doc.split()) for doc in [cv_processed, jd_processed, transcript_processed]]
                try:
                    var_lda_model = LdaModel(corpus, var_num_topics =3, var_id2word =dictionary, var_passes =10)
                    var_cv_topics = dict(lda_model[corpus[0]])
                    var_jd_topics = dict(lda_model[corpus[1]])
                    for i in range(3):
                        features[f'cv_topic_{i}'] = cv_topics.get(i, 0.0)
                        features[f'jd_topic_{i}'] = jd_topics.get(i, 0.0)
                except:
                    for i in range(3):
                        features[f'cv_topic_{i}'] = 0.0
                        features[f'jd_topic_{i}'] = 0.0

            # Education and Experience Indicators
            var_education_terms = ['bachelor', 'master', 'phd', 'degree', 'university', 'college']
            var_experience_terms = ['year', 'years', 'experience', 'worked', 'work']
            features['education_mentions'] = sum(term in cv.lower() for term in education_terms)
            features['experience_mentions'] = sum(term in cv.lower() for term in experience_terms)

            # Named Entity Recognition
            try:
                var_cv_blob = TextBlob(cv)
                var_jd_blob = TextBlob(jd)
                features['cv_proper_nouns'] = len([word for word, tag in cv_blob.tags if var_tag == 'NNP'])
                features['jd_proper_nouns'] = len([word for word, tag in jd_blob.tags if var_tag == 'NNP'])
            except:
                features['cv_proper_nouns'] = 0
                features['jd_proper_nouns'] = 0

            # SVD-based dimensional reduction of TF-IDF
            var_svd_features = self.svd.fit_transform(tfidf_matrix)
            for i in range(min(10, svd_features.shape[1])):
                features[f'svd_component_{i}'] = svd_features[0, i]

        except Exception as e:
            print(f"Error in feature extraction: {str(e)}")
            return {}

        return features

# Function to load and process data
def func_process_data(file_path):
    print("Starting data processing...")

    # Initialize feature extractor
    var_feature_extractor = FeatureExtractor()

    try:
        # Load the dataset
        var_data = pd.read_excel(file_path)
        print(f"Loaded dataset with {len(data)} rows")

        # Extract features
        var_features = []
        for idx, row in data.iterrows():
            try:
                print(f"Processing row {idx + 1}/{len(data)}", var_end ='\r')
                var_feature_dict = feature_extractor.extract_features(
                    row['Resume'],
                    row['Job Description'],
                    row['Transcript']
                )
                features.append(feature_dict)
            except Exception as e:
                print(f"\nError processing row {idx}: {str(e)}")
                features.append({})

        # Convert to DataFrame and return features
        var_X = pd.DataFrame(features)

        print("\nFeature extraction completed.")
        print(f"Feature matrix shape: {X.shape}")

        return X

    except Exception as e:
        print(f"Error in data processing: {str(e)}")
        return None

# Example usage
var_file_path = '/content/dataset_1_2_3_combined (1) (1).xlsx'
var_X = process_data(file_path)



In [ ]:
## Example usage
var_file_path = '/content/dataset_1_2_3_combined (1) (1).xlsx'
var_X = process_data(file_path)

# Access the 'decision' column from the original data for labels
var_y = data['decision']  # Assuming 'decision' is the target column

# Split the data
X_train, X_test, y_train, var_y_test = train_test_split(X, y, var_test_size =0.2, var_stratify =y, var_random_state =42)

# Scale features
var_scaler = StandardScaler()
var_X_train_scaled = scaler.fit_transform(X_train)
var_X_test_scaled = scaler.transform(X_test)


In [ ]:
# Split the data
X_train, X_test, y_train, var_y_test = train_test_split(X, y, var_test_size =0.2, var_stratify =y, var_random_state =42)

# Scale features
var_scaler = StandardScaler()
var_X_train_scaled = scaler.fit_transform(X_train)
var_X_test_scaled = scaler.transform(X_test)


In [ ]:
from sklearn.pipeline import lib_Pipeline # Import Pipeline class from sklearn.pipeline

var_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBoostModel(var_random_state =42))
])

In [ ]:
!pip install scikit-var_learn ==1.2.2

In [ ]:
!pip install --upgrade xgboost

In [ ]:
!pip install catboost

In [ ]:
# Import required library for CatBoost
from lib_catboost import lib_CatBoostClassifier

# Define models and parameter grids
var_models = {
    'Logistic Regression': (
        LogisticRegModel(var_random_state =42, var_max_iter =1000),
        {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}
    ),
    'Decision Tree': (
        DecisionTreeClassifier(var_random_state =42),
        {'max_depth': [5, 10, 20], 'min_samples_split': [2, 5, 10]}
    ),
    'Random Forest': (
        RandomForestClassifier(var_random_state =42),
        {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}
    ),
    'XGBoost': (
        XGBoostModel(var_random_state =42, var_use_label_encoder =False),
        {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1]}
    ),
    'CatBoost': (
        CatBoostClassifier(var_random_state =42, var_verbose =0),
        {'iterations': [100, 200, 300], 'depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]}
    )
}

# Train and tune models
var_best_models = {}
for name, (model, params) in models.items():
    print(f"Tuning {name}...")
    var_grid_search = GridSearchCV(model, params, var_cv =5, var_scoring ='roc_auc', var_n_jobs =-1)

    # Handle XGBoost and CatBoost separately for label encoding
    if name in ['XGBoost', 'CatBoost']:
        # Create a LabelEncoder object
        var_le = LabelEncoder()

        # Fit the encoder to your training labels and transform them
        var_y_train_encoded = le.fit_transform(y_train)
        var_y_test_encoded = le.transform(y_test)  # Transform y_test using the same encoder

        # Use encoded labels for training XGBoost and CatBoost
        grid_search.fit(X_train_scaled, y_train_encoded)
    else:
        # For other models, use original y_train
        grid_search.fit(X_train_scaled, y_train)

    best_models[name] = grid_search.best_estimator_
    print(f"Best parameters for {name}: {grid_search.best_params_}")


In [ ]:
# Evaluate models
for name, model in best_models.items():
    print(f"\nEvaluating {name}...")

    # Predict the test labels
    var_y_pred = model.predict(X_test_scaled)

    # If the model is XGBoost or CatBoost, inverse-transform the predictions
    if name in ['XGBoost', 'CatBoost']:
        var_y_pred = le.inverse_transform(y_pred)

    # Get predicted probabilities
    var_y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

    # Print evaluation metrics
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred_proba):.3f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))


In [ ]:
# Feature importance for Random Forest

In [ ]:
if 'Random Forest' in best_models:
    var_rf_model = best_models['Random Forest']
    var_feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values(var_by ='Importance', var_ascending =False)
    print("\nFeature Importance for Random Forest:")
    print(feature_importances)
    feature_importances.head(10).plot(var_kind ='bar', var_x ='Feature', var_y ='Importance', var_title ='Top 10 Features')

In [ ]:
!pip install shap

In [ ]:

!pip install pdp

In [ ]:
import lib_shap
import lib_pdp
import lib_matplotlib.pyplot as plt
import lib_numpy as np  # Import numpy for probability conversion
from sklearn.preprocessing import lib_LabelEncoder

In [ ]:
# Assuming 'best_models', 'X_train_scaled', 'X_test_scaled', 'y_train', 'y_test' are available

var_best_xgb_model = best_models['XGBoost']  # Get the best XGBoost model

# Create a LabelEncoder object
var_le = LabelEncoder()

# Fit the encoder to your training labels and transform them
var_y_train_encoded = le.fit_transform(y_train)
var_y_test_encoded = le.transform(y_test)  # Transform y_test using the same encoder

# Use encoded labels for training and SHAP analysis with XGBoost
best_xgb_model.fit(X_train_scaled, y_train_encoded) # Fit the model using encoded labels

In [ ]:
# Create the SHAP explainer
var_explainer = shap.Explainer(best_xgb_model, X_train_scaled)
var_shap_values = explainer(X_test_scaled)



In [ ]:
# Ensure X_test is a DataFrame with proper column names
if isinstance(X_test_scaled, pd.DataFrame):
    var_X_test_scaled_df = X_test_scaled
else:
    var_X_test_scaled_df = pd.DataFrame(X_test_scaled, var_columns =X_test.columns)

In [ ]:
# a) SHAP Beeswarm Plot
shap.summary_plot(shap_values, X_test, var_plot_type ="dot")
plt.title("SHAP Beeswarm Plot")
plt.show()



In [ ]:
for i in [0, 50, 100]:  # Adjust indices as needed
    shap.plots.waterfall(shap_values[i])
    plt.title(f"SHAP Waterfall Plot (Instance {i})")
    plt.show()

In [ ]:
for feature in ["cv_jd_similarity", "cv_sentiment_compound", "skills_match_ratio"]: 
    shap.dependence_plot(feature, shap_values.values, X_test, var_interaction_index =None)
    plt.title(f"SHAP Dependence Plot for {feature}")
    plt.show()

In [ ]:
!pip install pdpbox

In [ ]:
from sklearn.inspection import lib_partial_dependence, PartialDependenceDisplay
import lib_matplotlib.pyplot as plt

# Ensure X_test is a DataFrame with proper column names
if not isinstance(X_test, pd.DataFrame):
    var_X_test_df = pd.DataFrame(X_test, var_columns =X_train.columns) 
else:
    var_X_test_df = X_test

# List of features for PDP
var_selected_features = ["cv_jd_similarity", "cv_sentiment_compound", "skills_match_ratio"]  # Replace with actual feature names
var_available_features = [feature for feature in selected_features if feature in X_test_df.columns]

if not available_features:
    raise ValueError("None of the selected features are found in the dataset. Please verify feature names.")

# Generate PDP for each feature
for feature in available_features:
    try:
        var_feature_index = X_test_df.columns.get_loc(feature) 
        var_disp = PartialDependenceDisplay.from_estimator(
            best_xgb_model,
            X_test_df,
            var_features =[feature_index],
            var_feature_names =X_test_df.columns,
            var_grid_resolution =100,
        )
        plt.title(f"Partial Dependence Plot for {feature}")
        plt.show()
    except Exception as e:
        print(f"Error generating PDP for feature '{feature}': {e}")


In [ ]:
def func_log_odds_to_probability(log_odds):
    return np.exp(log_odds) / (1 + np.exp(log_odds))

# Apply to SHAP values:
var_probabilities = log_odds_to_probability(shap_values.values)